<a href="https://colab.research.google.com/github/yasirfaizahmed/Text-Classification-models/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
dataset = load_dataset("imsoumyaneel/sentiment-analysis-llama2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/173M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/598298 [00:00<?, ? examples/s]

In [ ]:
# converting the training dataset to a dataframe
df = pd.DataFrame(dataset.get("train"))

In [ ]:
max_length_manual = 0
i = 0
for idx, sentence in enumerate(df["sentence"]):
  if len(sentence) > max_length_manual:
    max_length_manual = len(sentence)
    i = idx

# getting the max length of a sentence
max_length = df["sentence"].apply(len).max()


In [ ]:
# mapping the labels to numbers
df["mapped_label"] = df["label"].map({'joy': 0, 'neutral': 1, 'sadness': 2, 'anger': 3, 'fear': 4, 'love': 5, 'surprise': 6})
labels = df["mapped_label"].astype("float32").values

In [ ]:
nltk.download('stopwords')
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stopwords(sentence):
  return re.sub(r'[^\w\s]','', sentence)

In [ ]:
df['sentence'] = df["sentence"].map(remove_stopwords)

In [ ]:
# map or apply can be used here to apply the function to a series or a dataframe
def remove_stopwords(sentence):
  new_sentence = ""
  for word in sentence.split(" "):
    if word not in stopwords:
      new_sentence += " " + word
  # print(new_sentence)
  return new_sentence

df['sentence'] = df['sentence'].map(remove_stopwords)

In [ ]:
# tokenize the words in sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(df["sentence"])
tokenized_sentences = tokenizer.texts_to_sequences(df["sentence"])

In [ ]:
print("0th text: ", tokenizer.sequences_to_texts(tokenized_sentences)[0])
print("0th token: ", tokenized_sentences[0])
print("actual 0th text: ", df['sentence'][0])

0th text:  ill throw garbage
0th token:  [77, 1131, 2991]
actual 0th text:   Ill throw garbage 


In [ ]:
# pad the smaller sentences
small_max_length = 200
vocab_size = len(tokenizer.word_index)
padded_tokenized_sentences = tf.keras.preprocessing.sequence.pad_sequences(tokenized_sentences,
                                                                 maxlen=200,
                                                                 padding='post')

In [ ]:
# modeling
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16,
                              embeddings_initializer='uniform'),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_tokenized_sentences, labels, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(labels.shape)

(418808, 200)
(179490, 200)
(418808,)
(179490,)
(598298,)


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=7, validation_data=(X_test, y_test))

Epoch 1/10
 4655/59830 ━━━━━━━━━━━━━━━━━━━━ 13:23 15ms/step - accuracy: 0.2606 - loss: 1.7646

KeyboardInterrupt: 